In [2]:
base_wiki_url = "https://overwatch.fandom.com/wiki/"

# Too lazy to scrap the page, just copy-pasted the hero names from here:
active_hero_list = [
    "Baptiste‎",
    "Bastion‎",
    "Brigitte‎",
    "Cassidy‎",
    "D.Va‎",
    "Doomfist‎",
    "Echo‎",
    "Genji‎",
    "Hanzo‎",
    "Junkrat‎",
    "Lifeweaver‎",
    "Lúcio‎",
    "Mei‎",
    "Mercy‎",
    "Moira‎",
    "Orisa‎",
    "Pharah‎",
    "Ramattra‎",
    "Reaper‎",
    "Reinhardt‎",
    "Roadhog‎",
    "Sigma‎",
    "Soldier: 76‎",
    "Sombra‎",
    "Symmetra‎",
    "Torbjörn‎",
    "Tracer‎",
    "Widowmaker‎",
    "Winston‎",
    "Wrecking Ball‎",
    "Zarya‎",
    "Zenyatta‎",
]

In [3]:
import requests
import time
from  bs4 import BeautifulSoup


def parse_ability_body_row(attribute):
    if len(attribute.contents) == 2:
        return {attribute.contents[0].text:attribute.contents[1].text}
    return {}
    
def parse_ability_box(soup):
    ability_data = {}
    # Parse main sections of ability box
    ability_header = soup.find("div", {"class": "abilityHeader"})
    summary_info_and_image = soup.find("div", {"class": "summaryInfoAndImage"})
    ability_body = list(soup.contents)[2]

    # Parse ability name
    ability_name = ability_header.text

    # Parse ability keybind
    keybind = ability_header.find("span", {"title": "Hotkey"})
    if keybind:
        if keybind.text:
            keybind = keybind.text
        else:
            keybind = keybind.find("img")['alt']    
        ability_data = {"keybind": keybind}    
        ability_name = ability_header.text.replace(keybind, "")
    
    # Parse attributes in ability body
    for attribute in ability_body.contents:
        attribute_data = parse_ability_body_row(attribute)
        ability_data.update(attribute_data)
    # TODO: Parse the rest of the ability_box

    data = {ability_name: ability_data}
    return data

def get_abilities(character: str):
    # Generate hero-specific URL
    url = base_wiki_url + character.replace(" ", "_")
    print(f"Loading URL: {url}")
    # Query and parse URL
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    # content_soup = soup.find("div", {"class": "mw-parser-output"})
    ability_box_list = soup.find_all("div", {"class": "ability_box"})
    ability_data = {}
    for ability_box in ability_box_list:
        ability_data.update(parse_ability_box(ability_box))
    return ability_data

get_abilities("Reaper")
# for hero in active_hero_list:
#     get_abilities(hero)

Loading URL: https://overwatch.fandom.com/wiki/Reaper


{'The Reaping': {'Healing:': ' 35% of damage dealt'},
 'Hellfire Shotguns': {'keybind': 'Key mouse 1',
  'Damage:': ' 5.4 – 1.62 per pellet108 – 36 per shot',
  'Falloff range:': ' 10 – 20 meters',
  'Headshot:': ' ✓',
  'Rate of fire:': ' 0.496 seconds recovery',
  'Bullets per shot:': ' 20',
  'Ammo per shot:': ' 1',
  'Ammo:': ' 8',
  'Reload time:': ' 1.5 seconds animation',
  'Spread angle:': ' 7'},
 'Wraith Form': {'keybind': 'LSHIFT',
  'Duration:': ' 0.4 seconds min. duration3 seconds max. duration0.4 seconds recovery',
  'Max. range:': ' 24.75 meters',
  'Move. speed:': ' +50% buff'},
 'Shadow Step': {'keybind': 'E',
  'Cast time:': ' 0.992 s. + 1.008 s. invulnerability + 0.096 s. recovery',
  'Duration:': ' 2 seconds',
  'Max. range:': ' 35 meters'},
 'Death Blossom': {'keybind': 'Q',
  'Damage:': ' 170 damage per second (per target)',
  'Headshot:': ' ✕',
  'Duration:': ' 3 seconds',
  'Area of effect:': ' 8 meter radius',
  'Move. speed:': ' -50% penalty'}}